## Intro

The idea of this notebook was to figure out, how to use the potential of the cluster and parallelize the training of different ML models by using spark and hyperopt. Hyperopt is a general-purpose library, which can optimize any function that has parameters here is will use it for the loss function of the prediction (rmse).

I understood that in the end a monthly aggredation is needed but i will work with daily data since the effect of weekdays and holidays is more visible. 

Simon B, 05.06.2023

In [0]:
%pip install prophet==1.1.4 hyperopt==0.2.7 sklearn mlflow

## Imports

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics         import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import ParameterGrid

from hyperopt import fmin, hp, tpe
from hyperopt import SparkTrials, STATUS_OK
from prophet import Prophet

import mlflow

## Get Input Parameters

In [0]:
# Get the name of the experiment 
# You must create the expirement by hand in the Databricks Machine Learning Experiments GUI
# Then copy the name of the experiment and paste it here

experimentPath = "/Users/simon.buse@ewz.ch/MittelfristPrognoseTest"

In [0]:
def logMetrics(test, predictions):
  '''
  Simple function to compute MAPE and RMSE metrics from test predictions and the test data for our models.
  The error metrics will be logged with MLflow.
  '''
  mapeValue = mean_absolute_percentage_error(test, predictions)
  rmseValue = mean_squared_error(test, predictions, squared=False)
  metrics = {"mape": mapeValue, "rmse": rmseValue}

  # MLflow will log the metrics to the currently active run
  mlflow.log_metrics(metrics)


def logParams(parameters):
  '''
  Parameters should be a dictionary with the form {"parameterName": "parameterValue"}
  '''
  for par in parameters:
    mlflow.log_params(par)

def reassign_outliers(pdf):
  """There is an extrem outlier in the data which is probably a mistake. I will reassign the value to it's neighbour."""

  for column in pdf.columns:
    
    outlier_loc = np.where(pdf[column] < np.mean(pdf[column])-3*np.std(pdf[column]))
    (pdf[column].values)[outlier_loc] = np.mean(pdf[column]) 

    print(f"Reassigned {len(outlier_loc)} values in the column {column}. These values where more than 3 sigma away from the mean.")

  return pdf


def resample_fix_ends(pdf,frequency):
  """
  The function resamples the data according to the sampling frequency. 
  Often the first and the last data-point are deviating a lot from the rest of the series.
  
  As a simple fix i will just delete the first and the last value if they deviate more than 20% to their neighbour. 
  """

  pdf = pdf.resample(frequency).sum(min_count=1) #"D,W,M"

  for column in pdf.columns:
    if pdf[column].iloc[0] < 0.8*pdf[column].iloc[1]:
      pdf = pdf.drop(pdf.index[0]) 
      #pdf.at[pdf.index[0],column]   = pdf[column].iloc[1] #this would assigne the value of the next day to the first day.

    if pdf[column].iloc[-1] < 0.8*pdf[column].iloc[-2]:
      pdf = pdf.drop(pdf.index[-1]) 
      #pdf.at[pdf.index[-1],column]  = pdf[column].iloc[-2] #this would assigne the value of the second last day to the last day.

  return pdf

## Prepare the Data

In [0]:
url = "https://data.stadt-zuerich.ch/dataset/ewz_stromabgabe_netzebenen_stadt_zuerich/download/ewz_stromabgabe_netzebenen_stadt_zuerich.csv"
pdf = pd.read_csv(url,index_col=None)

pdf["Timestamp"] =  pd.to_datetime(pdf['Timestamp'],utc=True)

pdf = pdf.set_index(pdf["Timestamp"])
pdf = resample_fix_ends(pdf,"D")
pdf = reassign_outliers(pdf)

pdf.index = pdf.index.tz_localize(None)  #Let's drop the timezone info to avoid warnings
pdf["ds"]= pdf.index

pdf

In [0]:
# Adding corona as one-off holidays https://facebook.github.io/prophet/docs/handling_shocks.html
corona = pd.DataFrame({
  'holiday': 'corona',
  'ds': pd.date_range(start='2020-03-01', end='2020-12-31', freq='D'),
  'lower_window': 0,
  'upper_window': (pd.to_datetime('2020-12-31') - pd.to_datetime('2020-03-01')).days,
})

## Define the Model and the Search Space

In [0]:
def train_prophet(pdfTrain,pdfTest,tracked_hyperparams, maxEvals=10,timeoutSec=5*60):
  """This function is just a wrapper for the hyperopt procedure."""



  def train(params):
    """
    This is our main training function which we pass to Hyperopt.
    It takes in hyperparameter settings, fits a model based on those settings,
    evaluates the model, and returns the loss. 
    """

    if tracked_hyperparams["corona"] == True:
      forecaster = Prophet(
        holidays                = corona,
        seasonality_mode        = tracked_hyperparams['seasonality_mode'],
        changepoint_prior_scale = params['changepoint_prior_scale'],
        seasonality_prior_scale = params['seasonality_prior_scale'],
        holidays_prior_scale    = params['holidays_prior_scale'],
        changepoint_range       = params['changepoint_range'])
      
    else:
      forecaster = Prophet(
        seasonality_mode        = tracked_hyperparams['seasonality_mode'],
        changepoint_prior_scale = params['changepoint_prior_scale'],
        seasonality_prior_scale = params['seasonality_prior_scale'],
        holidays_prior_scale    = params['holidays_prior_scale'],
        changepoint_range       = params['changepoint_range'])

    if tracked_hyperparams["holidays"] != None:   
      forecaster.add_country_holidays(country_name=tracked_hyperparams["holidays"])

    forecaster.fit(pdfTrain)  
    y_pred = forecaster.predict(pdfTest)

    rmse   = mean_squared_error(y_true=pdfTest.y.values, y_pred=y_pred.yhat.values, squared=False)  

    return {'loss': rmse, 'status': STATUS_OK, 'Trained_Model': forecaster}
   
  # Define the search space for Hyperopt.Prophets main parameter where found here
  # https://facebook.github.io/prophet/docs/diagnostics.html#hyperparameter-tuning 

  search_space = {
    "changepoint_prior_scale": hp.loguniform("changepoint_prior_scale", -6.9, -0.69), #equivalent to [0.001,0.5]
    "seasonality_prior_scale": hp.loguniform("seasonality_prior_scale", -6.9, 2.3),   #equivalent to [0.001, 10]
    "holidays_prior_scale":    hp.loguniform("holidays_prior_scale", -6.9, 2.3),      #equivalent to [0.001, 10]
    "changepoint_range":       hp.uniform("changepoint_range", 0.8,0.95)              #optional according to docs, default = 0.8               
  }

  # Select a search algorithm for Hyperopt to use.
  algo=tpe.suggest  # Tree of Parzen Estimators, a Bayesian method

  # Distribute tuning across our Spark cluster
  spark_trials = SparkTrials(parallelism=4)

  best_hyperparameters = fmin(fn=train,space=search_space,algo=algo,trials=spark_trials,max_evals=maxEvals,timeout=timeoutSec)
  best_model = spark_trials.results[np.argmin([r['loss'] for r in spark_trials.results])]['Trained_Model']

  print(best_hyperparameters)
  return best_model,best_hyperparameters

## Train a single model on the sum

In [0]:
#train singel model on the sum of the two columns, let's add the tracking with Mlflow and comparison

sum_pdf = pdf.copy()
sum_pdf["total"] = pdf["Value_NE5"].values + pdf["Value_NE7"].values
sum_pdf = sum_pdf.drop(columns=["Value_NE5","Value_NE7"])
sum_pdf = sum_pdf.rename(columns={"total": "y"})
split = int(len(sum_pdf)*0.9)
pdf_train, pdf_test = sum_pdf.iloc[:split], sum_pdf.iloc[split:]


#define the hyperparams to track: holidays, additive or multiplicative, adding corona by hand or not, 
tracked_hyperparams =   {"seasonality_mode":['multiplicative','additive'], 
                         "corona":          [False, True],
                         "holidays":        [None, 'Switzerland']}

grid = ParameterGrid(tracked_hyperparams)

for evaluation_params in grid:
  
  # Create an MLflow run for each point in the grid space. 
  # The "with" syntax means the run will be closed once the following block of code is finished.
  with mlflow.start_run(run_name = "Prophet Model") as run:
    
    # Set a tag with the model type
    mlflow.set_tags({"Model": "Prophet"})

    #create, train and return model
    print(evaluation_params)
    best_model,best_hyperparameters = train_prophet(pdf_train,pdf_test, evaluation_params)

    prediction = best_model.predict(pdf_test) 

    # Log the metrics in MLflow 
    logMetrics(pdf_test.y.values, prediction.yhat.values)

    # Log hyperparameters
    logParams([best_hyperparameters,evaluation_params])





In [0]:
#plot the outcome of the model on the summed data.

f, axes = plt.subplots(2, 1,figsize=(18,8))

y_pred = best_model.predict(pdf_test)

axes[0].plot(y_pred.ds.values, y_pred.yhat.values, color="tab:red", label="forcast")
#axes[0].plot(pdf_train.ds.values, pdf_train.y.values, color="tab:blue", label="train")
axes[0].plot(pdf_test.ds.values, pdf_test.y.values, color="tab:orange", label="truth", alpha=0.5)
axes[0].legend()
axes[0].set_title("NE5 + NE7")
axes[0].set_ylabel("Last [kWh]")


xmin, xmax = axes[0].get_xlim()

axes[1].plot(pdf_test.ds,(pdf_test.y.values-y_pred.yhat.values)/(pdf_test.y.values)*100)
axes[1].set_xlim(xmin, xmax)
axes[1].set_ylabel("residual: True-Pred/True [%]")

plt.show()

In [0]:
forecast = best_model.predict(pdf)
best_model.plot(forecast)
fig = best_model.plot_components(forecast)

##Train 2 models for NE5 and NE7 separately

In [0]:
#train two separate models for the two series separately
best_induvidual_models = {}
best_model_params = {}

split = int(len(pdf)*0.9)
colum_names = ["Value_NE5","Value_NE7"]

for colum in colum_names:
  
  induvidual_pdf = pdf.copy()

  colum_to_drop = [x for x in colum_names if x != colum]
  induvidual_pdf = induvidual_pdf.drop(columns=colum_to_drop) #drop the unwanted columns
  induvidual_pdf = induvidual_pdf.rename(columns={colum: "y"})
  
  pdf_train, pdf_test = induvidual_pdf.iloc[:split], induvidual_pdf.iloc[split:]
  best_induvidual_models[colum],best_model_params[colum] = train_prophet(pdf_train,pdf_test)

In [0]:
#plot the outcome of two separate models

induvidual_predictions = []

for i,colum in enumerate(colum_names):

  pdf1 = pdf.copy()
  colum_to_drop = [x for x in colum_names if x != colum]
  pdf1 = pdf1.drop(columns=colum_to_drop)
  pdf1 = pdf1.rename(columns={colum: "y"})

  best_induvidual = best_induvidual_models[colum]

  pdf1_train, pdf1_test = pdf1.iloc[:split], pdf1.iloc[split:]
  y_pred = best_induvidual.predict(pdf1_test)
  
  f, axes = plt.subplots(2, 1,figsize=(18,8))

  axes[0].set_title(colum)
  axes[0].plot(y_pred.ds.values, y_pred.yhat.values, color="tab:red", label="forcast")
#  axes[0].plot(pdf1_train.ds.values, pdf1_train.y.values, color="tab:blue", label="train")
  axes[0].plot(pdf1_test.ds.values, pdf1_test.y.values, color="tab:orange", label="truth", alpha=0.5)
  axes[0].legend()


  xmin, xmax = axes[0].get_xlim()
  
  axes[1].plot(pdf1_test.ds,(pdf1_test.y.values-y_pred.yhat.values)/(pdf1_test.y.values)*100)
  axes[1].set_xlim(xmin, xmax)
  axes[1].set_ylabel("residual True-Pred/True [%]")

  
  plt.show()

  induvidual_predictions.append([y_pred.yhat.values,pdf1_test.y.values])




In [0]:
# Uncomment for a plot with components

# for i,colum in enumerate(colum_names):

#   pdf1 = pdf.copy()
#   colum_to_drop = [x for x in colum_names if x != colum]
#   pdf1 = pdf1.drop(columns=colum_to_drop)
#   pdf1 = pdf1.rename(columns={colum: "y"})

#   pdf1_train, pdf1_test = pdf1.iloc[:split], pdf1.iloc[split:]
#   best_induvidual = best_induvidual_models[colum]

#   forecast = best_induvidual.predict(pdf1)
#   print(colum)

#   best_induvidual.plot(forecast)
#   fig = best_induvidual.plot_components(forecast)


In [0]:
induvidual_predictions = np.array(induvidual_predictions)
prediction_on_sum = np.sum(induvidual_predictions,axis=0)

In [0]:
print("The rmse of the two separate models on the total is: {:.2e} ".format(mean_squared_error(y_true=prediction_on_sum[1], y_pred=prediction_on_sum[0], square_root=True)))
print("The rmse of the one induvidual model is:             {:.2e}".format(mean_squared_error(y_true=pdf_test.y.values, y_pred=y_pred.yhat.values, square_root=True)))

# It looks like one model is sufficient and we can simply operate on the sum of NE5 and NE7. 

## Notes

In [0]:
# resample to monthly if necessary

# f, axes = plt.subplots(1, 1,figsize=(18,8))

# y_pred    = resample_fix_ends(induvidual_predictions[1][0],"M")+resample_fix_ends(induvidual_predictions[0][0],"M").values
# pdf_train = resample_fix_ends(induvidual_predictions[1][1],"M")+resample_fix_ends(induvidual_predictions[0][1],"M").values         
# pdf_test  = resample_fix_ends(induvidual_predictions[1][2],"M")+resample_fix_ends(induvidual_predictions[0][2],"M").values         

# plt.plot(y_pred, color="tab:red",label="forcast")
# plt.plot(pdf_train, color="tab:green",label="train")
# plt.plot(pdf_test, color="tab:orange", label="test")
# plt.title("Sum of NE5+NE7")
# plt.legend()
# plt.show()

# #evaluate how good the model is--> rmse
# rmse   = mean_squared_error(y_true=pdf_test, y_pred=y_pred, square_root=True) 
# print(rmse)

In [0]:

# space = hp.choice('classifier_type', [
#     {
#         'type': 'naive_bayes',
#     },
#     {
#         'type': 'svm',
#         'C': hp.lognormal('svm_C', 0, 1),
#         'kernel': hp.choice('svm_kernel', [
#             {'ktype': 'linear'},
#             {'ktype': 'RBF', 'width': hp.lognormal('svm_rbf_width', 0, 1)},
#             ]),
#     },
#     {
#         'type': 'dtree',
#         'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
#         'max_depth': hp.choice('dtree_max_depth',
#             [None, hp.qlognormal('dtree_max_depth_int', 3, 1, 1)]),
#         'min_samples_split': hp.qlognormal('dtree_min_samples_split', 2, 1, 1),
#     },
#     ])